In [1]:
import numpy as np
import svox2
import torch
from k3d.colormaps import matplotlib_color_maps
import open3d as o3d
from tqdm import tqdm
from scipy.io import loadmat
import k3d
from opt.util.dataset import datasets
import os

np.set_printoptions(suppress=True)
torch.set_printoptions(sci_mode=False)

/home/tw554/plenoxels/svox2/utils.py:42: UserWarning: CUDA extension svox2.csrc could not be loaded! Operations will be slow.
Please do not import svox in the svox2 source directory.
  warn("CUDA extension svox2.csrc could not be loaded! " +


In [2]:
path = "/home/tw554/plenoxels/opt/ckpt/sparsify/000/ckpt.npz"
# path = "opt/ckpt/dtu_scan63/ex_3_no_alpha/"
# path = "opt/ckpt/dtu_scan63/no_surface_init_long_init_high_lap/"
# path = "opt/ckpt/dtu_scan63/norm_loss_con_low/"
# path = "opt/ckpt/udf_debug_3/ckpt_00500.npz"

if os.path.isfile(path):
    grid = svox2.SparseGrid.load(path, device='cuda')
else:
    grid = svox2.SparseGrid.load(path + "ckpt.npz", device='cuda')

surface_data = grid.surface_data
alpha_data = grid.density_data
grid.step_id

100000

# Inspect non-empty voxels

In [10]:
# l = [2,4,3]
# l = [ 74, 64, 101]
# l = [ 74, 64, 104]

def get_voxel_surface_data(l, surface_data=surface_data):
    lx, ly, lz = l
    links000 = grid.links[lx, ly, lz]
    links001 = grid.links[lx, ly, lz + 1]
    links010 = grid.links[lx, ly + 1, lz]
    links011 = grid.links[lx, ly + 1, lz + 1]
    links100 = grid.links[lx + 1, ly, lz]
    links101 = grid.links[lx + 1, ly, lz + 1]
    links110 = grid.links[lx + 1, ly + 1, lz]
    links111 = grid.links[lx + 1, ly + 1, lz + 1]

    surfaces = [surface_data[links000,0], surface_data[links001,0], surface_data[links010,0], surface_data[links011,0], surface_data[links100,0], surface_data[links101,0], surface_data[links110,0], surface_data[links111,0]]

    return torch.stack(surfaces)

def get_voxel_alpha_data(l, alpha_data=alpha_data):
    lx, ly, lz = l
    links000 = grid.links[lx, ly, lz]
    links001 = grid.links[lx, ly, lz + 1]
    links010 = grid.links[lx, ly + 1, lz]
    links011 = grid.links[lx, ly + 1, lz + 1]
    links100 = grid.links[lx + 1, ly, lz]
    links101 = grid.links[lx + 1, ly, lz + 1]
    links110 = grid.links[lx + 1, ly + 1, lz]
    links111 = grid.links[lx + 1, ly + 1, lz + 1]

    alphas = [alpha_data[links000,0], alpha_data[links001,0], alpha_data[links010,0], alpha_data[links011,0], alpha_data[links100,0], alpha_data[links101,0], alpha_data[links110,0], alpha_data[links111,0]]

    return torch.stack(alphas)


In [18]:
fn = torch.nn.Softplus()

fn(get_voxel_surface_data([2,4,3]))

tensor([    5.6306,     0.2281,     4.7108,     4.7423,    10.1005,     0.0018,
            5.2897,     1.4869], device='cuda:0', grad_fn=<SoftplusBackward0>)

In [31]:
ls = np.stack(
    np.meshgrid(np.arange(grid.links.shape[0]-1),np.arange(grid.links.shape[1]-1),np.arange(grid.links.shape[2]-1))
    ).reshape(3, -1).T
# ls = np.stack(
#     np.meshgrid(np.arange(64, 128),np.arange(64, 128),np.arange(64, 128))
#     ).reshape(3, -1).T

render_ls = [ls[i] for i in range(ls.shape[0])]

for l in render_ls:
    lx, ly, lz = l
    links000 = grid.links[lx, ly, lz]
    links001 = grid.links[lx, ly, lz + 1]
    links010 = grid.links[lx, ly + 1, lz]
    links011 = grid.links[lx, ly + 1, lz + 1]
    links100 = grid.links[lx + 1, ly, lz]
    links101 = grid.links[lx + 1, ly, lz + 1]
    links110 = grid.links[lx + 1, ly + 1, lz]
    links111 = grid.links[lx + 1, ly + 1, lz + 1]
    alphas = [
        alpha_data[links000,0], 
        alpha_data[links001,0], 
        alpha_data[links010,0], 
        alpha_data[links011,0], 
        alpha_data[links100,0], 
        alpha_data[links101,0], 
        alpha_data[links110,0], 
        alpha_data[links111,0]
        ]
    alphas = torch.sigmoid(torch.stack(alphas)).cpu().detach().numpy()
    if np.min(alphas) >= 0.01:
        print(l)
        print(alphas)
        break


[6 1 5]
[0.9999989  1.         1.         1.         0.36254048 1.
 1.         1.        ]


# Visualize extracted pts

In [3]:
# to find voxel of pts
dset = datasets["auto"](
               "data/dtu/dtu_scan63",
               split="train",
               device="cuda",
               factor=1,
               n_images=-1)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 49/49 [00:04<00:00, 11.72it/s]


 Generating rays, scaling factor 1


/home/tw554/.conda/envs/voxel_sdf/lib/python3.8/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [4]:
# data_pcd = np.load(f'{path}/ckpt_eval_surf_trav/pts.npy')
data_pcd = np.load(f'/home/tw554/plenoxels/opt/ckpt/sparsify/000/ckpt_eval_surf_trav/ckpt_pts.npy')

lb = np.array([-300,-200,600])
ub = np.array([-50,0,800])

lb = np.array([-240,-300,500])
ub = np.array([100,100,900])

mask = (data_pcd > lb).all(axis=-1) & (data_pcd < ub).all(axis=-1)
pts = data_pcd[mask]
# pts = data_pcd
pts.shape



plt_points = k3d.points(pts,
                        color=0xff0000,
                        point_size=0.001,
                        shader="flat")


plot = k3d.plot(grid_visible=True,
                camera_auto_fit=True)
plot += plt_points
plot.display()

Output()

In [5]:
hit_pts = grid.world2grid(torch.tensor(dset.rescale2world(pts)))

print(hit_pts)

plt_points = k3d.points(hit_pts,
                        color=0xff0000,
                        point_size=0.001,
                        shader="flat")


plot = k3d.plot(grid_visible=True,
                camera_auto_fit=True)
plot += plt_points
plot.display()

tensor([[ 81.0907, 130.4815, 321.4035],
        [ 81.2725, 130.4778, 321.4713],
        [ 81.4480, 130.4709, 321.5504],
        ...,
        [214.3602, 291.1046, 349.1845],
        [217.9518, 288.0277, 352.2457],
        [218.5369, 287.3306, 352.9247]])


Output()

In [8]:
vex_pts = hit_pts.long()
links = grid.links[vex_pts[:,0], vex_pts[:,1], vex_pts[:,2]]

link_mask = links >= 0
vex_pts = vex_pts[link_mask]
links = links[link_mask]

surfs = grid.surface_data.data[links.long()]
density = grid.density_data.data[links.long()]

plt_points = k3d.points(positions=vex_pts.cpu().detach().numpy(),
#                         colors=colors,
                        point_size=0.5,
                        shader='3d',
#                         opacities=density.cpu().detach().numpy(),
#                         color_map=matplotlib_color_maps.Coolwarm,
                        attribute=density.clamp_max(200).cpu().detach().numpy()[:,0],
#                         color_range=[110.9, 110.8], 
#                         color=0x3f6bc5,
                       )

plot = k3d.plot()
plot += plt_points
plot.display()

Output()

In [6]:
vex_pts = torch.meshgrid(torch.arange(100, 200), torch.arange(100, 200), torch.arange(250, 350))
vex_pts = torch.stack(vex_pts).view(3, -1).T.to('cuda').long()
links = grid.links[vex_pts[:,0], vex_pts[:,1], vex_pts[:,2]]

link_mask = links >= 0
vex_pts = vex_pts[link_mask]
links = links[link_mask]

surfs = grid.surface_data.data[links.long()]
density = grid.density_data.data[links.long()]


plt_points = k3d.points(positions=vex_pts.cpu().detach().numpy(),
#                         colors=colors,
                        point_size=0.5,
                        shader='3d',
#                         opacities=density.cpu().detach().numpy(),
#                         color_map=matplotlib_color_maps.Coolwarm,
                        attribute=density.clamp_max(200).cpu().detach().numpy()[:,0],
#                         color_range=[110.9, 110.8], 
#                         color=0x3f6bc5,
                       )

plot = k3d.plot()
plot += plt_points
plot.display()

/home/tw554/.conda/envs/voxel_sdf/lib/python3.8/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "int64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Output()

In [14]:
density.mean()

tensor(4.2622, device='cuda:0')

tensor([[0.1144],
        [0.1144],
        [0.1144],
        ...,
        [0.0487],
        [0.1055],
        [0.0458]], device='cuda:0')

In [83]:
device='cuda'
def create_udf_surface(udfs, alphas, l, lv_sets, SURFACE_RANGE = 1.25, SAMPLE_NUM = 1e3):
    lx, ly, lz = l
    udf000, udf001, udf010, udf011, udf100, udf101, udf110, udf111 = \
        udfs

    alpha000, alpha001, alpha010, alpha011, alpha100, alpha101, alpha110, alpha111 = \
        alphas

    ys = torch.linspace(ly - SURFACE_RANGE, ly + 1 + SURFACE_RANGE, int(SAMPLE_NUM))
    zs = torch.linspace(lz - SURFACE_RANGE, lz + 1 + SURFACE_RANGE, int(SAMPLE_NUM))

    y_mesh, z_mesh = torch.meshgrid(ys,zs)
    yz = torch.vstack([y_mesh.flatten(), z_mesh.flatten()]).T

    pts = torch.zeros([int(SAMPLE_NUM**2), 3]).to(device)
    pts[:,1:] = yz

    wys, wzs = pts[:,1]-ly, pts[:,2]-lz
    c00 = udf000 * (1.-wzs) + udf001 * wzs
    c01 = udf010 * (1.-wzs) + udf011 * wzs
    c10 = udf100 * (1.-wzs) + udf101 * wzs
    c11 = udf110 * (1.-wzs) + udf111 * wzs
    c0 = c00 * (1.-wys) + c01 * wys
    c1 = c10 * (1.-wys) + c11 * wys
    # udf = c0 * (1.-wx) + c1 * wx = 0
    
    # find closest level set
    udf_avg = torch.mean(udfs)

    lv_dists = torch.abs(udf_avg - lv_sets)
    lv_set = lv_sets[lv_dists.min(axis=-1).indices]

    # print(f'level set is: {lv_set}')

    wxs =  (c0 -lv_set) / (c0 - c1)

    pts[:,0] = wxs.flatten() + lx


    # filter out points that are too far away
    min_dists = torch.min(torch.stack([torch.abs(pts[:, 0] - lx), torch.abs(pts[:, 0] - (lx+1))]), axis=0).values
    near_pt_ids = torch.arange(pts.shape[0])[(min_dists <= 2*SURFACE_RANGE) | ((lx <= pts[:, 0]) & (pts[:, 0] <= (lx+1)))]
    pts = pts[near_pt_ids,:]

    # filter out points with low alpha
    wa, wb = 1. - (pts - l), (pts - l)
        
    c00 = alpha000 * wa[:, 2:] + alpha001 * wb[:, 2:]
    c01 = alpha010 * wa[:, 2:] + alpha011 * wb[:, 2:]
    c10 = alpha100 * wa[:, 2:] + alpha101 * wb[:, 2:]
    c11 = alpha110 * wa[:, 2:] + alpha111 * wb[:, 2:]
    c0 = c00 * wa[:, 1:2] + c01 * wb[:, 1:2]
    c1 = c10 * wa[:, 1:2] + c11 * wb[:, 1:2]
    _alpha = c0 * wa[:, :1] + c1 * wb[:, :1]
    # post sigmoid activation
    _alpha = torch.sigmoid(_alpha)
    
    th = 0.01
    
    pts = pts[_alpha[:, 0] > th]
    _alpha = _alpha[_alpha[:, 0] > th]

    return pts, _alpha

def find_normal_center(surfaces):
    surface000, surface001, surface010, surface011, surface100, surface101, surface110, surface111 = \
        surfaces.unbind(-1)
    x = _x = y = _y = z = _z = 0.5

    c00 = surface000 * _z + surface001 * z
    c01 = surface010 * _z + surface011 * z
    c10 = surface100 * _z + surface101 * z
    c11 = surface110 * _z + surface111 * z
    c0 = c00 * _y + c01 * y
    c1 = c10 * _y + c11 * y
    # surface = c0 * _x + c1 * x

    ds_dx = c1 - c0
    ds_dy = (c01-c00)*_x + (c11-c10)*x

    dc00_dz = surface001 - surface000
    dc01_dz = surface011 - surface010
    dc10_dz = surface101 - surface100
    dc11_dz = surface111 - surface110
    ds_dz = (dc00_dz * _y + dc01_dz * y)*_x + (dc10_dz * _y + dc11_dz * y)*x

    normals = torch.stack([ds_dx, ds_dy, ds_dz], dim=-1)
    normals = normals / torch.norm(normals, dim=-1, keepdim=True)
    return normals

with torch.no_grad():

    links = grid.links
#     surface_data = grid.surface_data
#     alpha_data = grid.density_data
    lv_sets = grid.level_set_data

    ls = torch.stack(
        torch.meshgrid(torch.arange(links.shape[0]-1),torch.arange(links.shape[1]-1),torch.arange(links.shape[2]-1))
        ).reshape(3, -1).T.to(device)

    surface_pts = []
    surface_alphas = []
    surface_values = []

#     render_ls = [ls[i] for i in range(ls.shape[0])]
    render_ls = [
        [86,100,140], 
        [85,101,140],
        [85,100,140],
        [87,100,140],
        [86,101,140], 

#         [85,99,140],
#         [87,101,140],
#         [87,110,132],
        
#         [87,111,132],
        
#         [87,112,132],
        ]
    
    #74.7971, 100.2854, 132.5037
    #88.6427, 125.2865, 140.1453
    
    render_ls = np.reshape(np.stack(np.meshgrid(np.arange(80,85),np.arange(110,120), np.arange(132,140))), [3,-1]).T
    
#     render_ls = np.reshape(np.stack(np.meshgrid(np.arange(25,35),np.arange(15,25), np.arange(45,55))), [3,-1]).T

    for l in tqdm(render_ls):
        tl = torch.tensor(l).long().to(device)
        if (tl < 0).any() or (tl >= torch.tensor(links.shape).to(device) - 1).any():
            continue 
        lx, ly, lz = tl
        links000 = links[lx, ly, lz]
        links001 = links[lx, ly, lz + 1]
        links010 = links[lx, ly + 1, lz]
        links011 = links[lx, ly + 1, lz + 1]
        links100 = links[lx + 1, ly, lz]
        links101 = links[lx + 1, ly, lz + 1]
        links110 = links[lx + 1, ly + 1, lz]
        links111 = links[lx + 1, ly + 1, lz + 1]

        surfaces = torch.tensor([surface_data[links000,0], surface_data[links001,0], surface_data[links010,0], surface_data[links011,0], surface_data[links100,0], surface_data[links101,0], surface_data[links110,0], surface_data[links111,0]]).to(device)
        alphas = torch.tensor([alpha_data[links000,0], alpha_data[links001,0], alpha_data[links010,0], alpha_data[links011,0], alpha_data[links100,0], alpha_data[links101,0], alpha_data[links110,0], alpha_data[links111,0]]).to(device)

    #     print(f'udf Raw: {surfaces}')

        v_pts, pt_alphas = create_udf_surface(surfaces, alphas, tl, lv_sets, SURFACE_RANGE=0, SAMPLE_NUM=50)

        surface_pts.append(v_pts)
        surface_alphas.append(pt_alphas)
        surface_values.append(surfaces)

    surface_pts = torch.cat(surface_pts).cpu().detach().numpy()
    surface_alphas = torch.cat(surface_alphas).cpu().detach().numpy()
    surface_values = torch.stack(surface_values,dim=0).cpu().detach().numpy()


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 400/400 [00:02<00:00, 186.54it/s]


In [84]:
plt_points = k3d.points(positions=surface_pts,
                        point_size=0.02,
                        shader='3d',
#                         opacities=surface_alphas,
#                         color_map=matplotlib_color_maps.Coolwarm,
                        attribute=surface_alphas,
#                         color=0x3f6bc5,
                       )

plot = k3d.plot()
plot += plt_points
norms = find_normal_center(torch.tensor(surface_values))


vectors = k3d.vectors(origins=np.array(render_ls) + 0.5,
            vectors=norms.cpu().detach().numpy(),
           )

# plot += vectors

ls = np.array(render_ls)
vertices = np.array([
    [0,0,0],
    [0,0,1],
    [0,1,0],
    [0,1,1],
    [1,0,0],
    [1,0,1],
    [1,1,0],
    [1,1,1],
])

vers = ls[:,None,:] + vertices[None,:,:]
vers = np.reshape(vers, [-1,3])

color = np.reshape(surface_values-lv_sets.item(),[-1,1])
# print(norms)
# print(surface_values-lv_sets.item())
color = np.where(color > 0, 1., 0.)

mask = color[:,0]>0
# mask = color[:,0]>-100

plt_points = k3d.points(positions=vers[mask],
                        point_size=0.1,
                        shader='3d',
#                         color_map=matplotlib_color_maps.Coolwarm,
                        attribute=color[mask],
                       )

plot += plt_points
plot.display()

Output()

(8000, 1)

In [28]:
surface_values

array([[51.75892, 51.75892, 51.75892, ..., 51.75892, 51.75892, 51.75892],
       [51.75892, 51.75892, 51.75892, ..., 51.75892, 51.75892, 51.75892],
       [51.75892, 51.75892, 51.75892, ..., 51.75892, 51.75892, 51.75892],
       ...,
       [51.75892, 51.75892, 51.75892, ..., 51.75892, 51.75892, 51.75892],
       [51.75892, 51.75892, 51.75892, ..., 51.75892, 51.75892, 51.75892],
       [51.75892, 51.75892, 51.75892, ..., 51.75892, 51.75892, 51.75892]],
      dtype=float32)

In [6]:
torch.sigmoid((torch.tensor(surface_values, device=device) + alpha_lv_sets[0] - grid.level_set_data[0]))

tensor([[    0.5484,     0.4587,     0.4145,  ...,     0.6422,     0.3693,
             0.6105],
        [    0.4587,     0.3975,     0.7804,  ...,     0.4637,     0.6105,
             0.5192],
        [    0.3975,     0.4669,     0.6535,  ...,     0.5674,     0.5192,
             0.4874],
        ...,
        [    0.0000,     0.0000,     0.0000,  ...,     0.0000,     0.0000,
             0.0000],
        [    0.0000,     0.0000,     0.0000,  ...,     0.0000,     0.0000,
             0.0000],
        [    0.0000,     0.0000,     0.0000,  ...,     0.0000,     0.0000,
             0.0000]], device='cuda:0')

torch.Size([148258, 3])

In [41]:
reso = grid._grid_size().long()
coords = torch.meshgrid(torch.arange(reso[0]), torch.arange(reso[1]), torch.arange(reso[2]))
coords = torch.stack(coords).view(3, -1).T
links = grid.links[coords[:, 0], coords[:, 1], coords[:, 2]]
coords = coords[links>=0, :]
links = links[links>=0]
coords.shape

density, rgb, surface = grid._fetch_links(grid.links[coords[:, 0], coords[:, 1], coords[:, 2]])

alpha = 1 - torch.exp(-torch.clamp_min(density, 1e-6) * grid.opt.step_size)

fn = torch.nn.Softplus()
surface = fn(surface)

mask = alpha[:,0] > 0.00001
alpha = alpha[mask]
surface = surface[mask]
coords = coords[mask, :]
print(coords.shape)

alpha_lv_sets = [0.1]
alpha_lv_sets = torch.tensor(alpha_lv_sets, device=device)
alpha_lv_sets = torch.logit(alpha_lv_sets)
raw_alpha_data = torch.logit(alpha)
surface_data = raw_alpha_data.detach().clone()
surface = surface_data - alpha_lv_sets + grid.level_set_data[0]

attr = surface.clone()
attr[surface>grid.level_set_data] = 1
attr[surface<=grid.level_set_data] = 0

plt_points = k3d.points(positions=coords.cpu().detach().numpy(),
#                         colors=colors,
                        point_size=0.01,
                        shader='3d',
#                         opacities=surface_alphas,
                        color_map=matplotlib_color_maps.Coolwarm,
                        attribute=attr.cpu().detach().numpy()[:,0],
#                         color_range=[110.9, 110.8], 
#                         color=0x3f6bc5,
                       )

plot = k3d.plot()
plot += plt_points
plot.display()

torch.Size([10451, 3])


Output()